In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [8]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input
from keras.models import Model

def create_vgg_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [9]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator


def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=233):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            #zoom_range=0.05,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 32
        steps_train = len(curr_y)//bat_size
        
        
        model = create_vgg_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

import pickle
with open('../features/vgg_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/vgg_aug1_sub.csv', index=False)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.68390, saving model to best_m.h5
 - 7s - loss: 0.7123 - acc: 0.5145 - val_loss: 0.6839 - val_acc: 0.5486
Epoch 2/50
Epoch 00002: val_loss improved from 0.68390 to 0.68030, saving model to best_m.h5
 - 5s - loss: 0.6918 - acc: 0.5246 - val_loss: 0.6803 - val_acc: 0.5486
Epoch 3/50
Epoch 00003: val_loss did not improve
 - 5s - loss: 0.6925 - acc: 0.5213 - val_loss: 0.6912 - val_acc: 0.5486
Epoch 4/50
Epoch 00004: val_loss did not improve
 - 5s - loss: 0.6926 - acc: 0.5210 - val_loss: 0.6886 - val_acc: 0.5486
Epoch 5/50
Epoch 00005: val_loss did not improve
 - 5s - loss: 0.6887 - acc: 0.5469 - val_loss: 0.6903 - val_acc: 0.5486
Epoch 6/50
Epoch 00006: val_loss improved from 0.68030 to 0.62147, saving model to best_m.h5
 - 5s - loss: 0.6937 - acc: 0.5289 - val_loss: 0.6215 - val_acc: 0.6160
Epoch 7/50
Epoch 00007: val_loss improved from 0.62147 to 0.53605, saving model to best_m.h5
 - 5s - loss: 0.6334 - acc: 0.6334 - val_loss: 0.5360

Epoch 10/50
Epoch 00010: val_loss improved from 0.68844 to 0.68555, saving model to best_m.h5
 - 5s - loss: 0.7146 - acc: 0.5743 - val_loss: 0.6855 - val_acc: 0.5287
Epoch 11/50
Epoch 00011: val_loss improved from 0.68555 to 0.59851, saving model to best_m.h5
 - 5s - loss: 0.6820 - acc: 0.5252 - val_loss: 0.5985 - val_acc: 0.5287
Epoch 12/50
Epoch 00012: val_loss improved from 0.59851 to 0.56404, saving model to best_m.h5
 - 4s - loss: 0.6205 - acc: 0.6103 - val_loss: 0.5640 - val_acc: 0.6858
Epoch 13/50
Epoch 00013: val_loss improved from 0.56404 to 0.51958, saving model to best_m.h5
 - 5s - loss: 0.5822 - acc: 0.6719 - val_loss: 0.5196 - val_acc: 0.7406
Epoch 14/50
Epoch 00014: val_loss improved from 0.51958 to 0.46240, saving model to best_m.h5
 - 5s - loss: 0.5706 - acc: 0.6950 - val_loss: 0.4624 - val_acc: 0.8204
Epoch 15/50
Epoch 00015: val_loss improved from 0.46240 to 0.42381, saving model to best_m.h5
 - 4s - loss: 0.4936 - acc: 0.7842 - val_loss: 0.4238 - val_acc: 0.8454
Epoc

Epoch 18/50
Epoch 00018: val_loss did not improve
 - 5s - loss: 0.2473 - acc: 0.9020 - val_loss: 0.2462 - val_acc: 0.8903
Epoch 19/50
Epoch 00019: val_loss improved from 0.24220 to 0.22668, saving model to best_m.h5
 - 5s - loss: 0.2586 - acc: 0.8839 - val_loss: 0.2267 - val_acc: 0.9127
Epoch 20/50
Epoch 00020: val_loss did not improve
 - 5s - loss: 0.2528 - acc: 0.8913 - val_loss: 0.3019 - val_acc: 0.8529
Epoch 21/50
Epoch 00021: val_loss did not improve
 - 5s - loss: 0.2343 - acc: 0.9046 - val_loss: 0.2722 - val_acc: 0.8628
Epoch 22/50
Epoch 00022: val_loss did not improve
 - 4s - loss: 0.2463 - acc: 0.9076 - val_loss: 0.2725 - val_acc: 0.8978
Epoch 23/50
Epoch 00023: val_loss did not improve
 - 5s - loss: 0.2296 - acc: 0.9062 - val_loss: 0.2289 - val_acc: 0.8878
Epoch 24/50
Epoch 00024: val_loss did not improve
 - 4s - loss: 0.2827 - acc: 0.8843 - val_loss: 0.2701 - val_acc: 0.8703
Epoch 25/50
Epoch 00025: val_loss did not improve
 - 4s - loss: 0.2351 - acc: 0.9051 - val_loss: 0.227

Epoch 32/50
Epoch 00032: val_loss improved from 0.33626 to 0.33277, saving model to best_m.h5
 - 5s - loss: 0.3002 - acc: 0.8795 - val_loss: 0.3328 - val_acc: 0.8479
Epoch 33/50
Epoch 00033: val_loss improved from 0.33277 to 0.32432, saving model to best_m.h5
 - 5s - loss: 0.3012 - acc: 0.8710 - val_loss: 0.3243 - val_acc: 0.8728
Epoch 34/50
Epoch 00034: val_loss improved from 0.32432 to 0.31033, saving model to best_m.h5
 - 5s - loss: 0.3123 - acc: 0.8784 - val_loss: 0.3103 - val_acc: 0.8678
Epoch 35/50
Epoch 00035: val_loss did not improve
 - 4s - loss: 0.2673 - acc: 0.9065 - val_loss: 0.3118 - val_acc: 0.8579
Epoch 36/50
Epoch 00036: val_loss did not improve
 - 4s - loss: 0.2880 - acc: 0.8868 - val_loss: 0.3200 - val_acc: 0.8529
Epoch 37/50
Epoch 00037: val_loss improved from 0.31033 to 0.30301, saving model to best_m.h5
 - 5s - loss: 0.2708 - acc: 0.8964 - val_loss: 0.3030 - val_acc: 0.8828
Epoch 38/50
Epoch 00038: val_loss did not improve
 - 4s - loss: 0.2915 - acc: 0.8927 - val_l

In [10]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_incept_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')
    
    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')
    
    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')

    x = Conv2D(64, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

model model


In [11]:

def kfold_train(fold_cnt=3,rnd=233):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            #zoom_range=0.05,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 32
        steps_train = len(curr_y)//bat_size
        
        
        model = create_incept_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)


with open('../features/incept_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/incept_aug1_sub.csv', index=False)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.63164, saving model to best_m.h5
 - 8s - loss: 0.5906 - acc: 0.6877 - val_loss: 0.6316 - val_acc: 0.7307
Epoch 2/50
Epoch 00002: val_loss improved from 0.63164 to 0.62645, saving model to best_m.h5
 - 4s - loss: 0.4256 - acc: 0.8009 - val_loss: 0.6265 - val_acc: 0.5486
Epoch 3/50
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.3654 - acc: 0.8246 - val_loss: 1.5543 - val_acc: 0.5486
Epoch 4/50
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.3091 - acc: 0.8465 - val_loss: 1.2458 - val_acc: 0.6334
Epoch 5/50
Epoch 00005: val_loss improved from 0.62645 to 0.29739, saving model to best_m.h5
 - 3s - loss: 0.3323 - acc: 0.8460 - val_loss: 0.2974 - val_acc: 0.8653
Epoch 6/50
Epoch 00006: val_loss did not improve
 - 3s - loss: 0.3283 - acc: 0.8631 - val_loss: 0.6132 - val_acc: 0.7007
Epoch 7/50
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.3013 - acc: 0.8719 - val_loss: 0.6242 - val_acc: 0.7007
Epoch 8/50
Epoch 00008: v

 - 3s - loss: 0.2487 - acc: 0.8961 - val_loss: 0.2932 - val_acc: 0.8554
Epoch 15/50
Epoch 00015: val_loss did not improve
 - 3s - loss: 0.2143 - acc: 0.9099 - val_loss: 0.3132 - val_acc: 0.8928
Epoch 16/50
Epoch 00016: val_loss did not improve
 - 3s - loss: 0.2399 - acc: 0.8904 - val_loss: 0.3391 - val_acc: 0.8354
Epoch 17/50
Epoch 00017: val_loss improved from 0.29322 to 0.28534, saving model to best_m.h5
 - 3s - loss: 0.2611 - acc: 0.8936 - val_loss: 0.2853 - val_acc: 0.9052
Epoch 18/50
Epoch 00018: val_loss improved from 0.28534 to 0.27287, saving model to best_m.h5
 - 3s - loss: 0.2127 - acc: 0.9144 - val_loss: 0.2729 - val_acc: 0.8778
Epoch 19/50
Epoch 00019: val_loss did not improve
 - 4s - loss: 0.2237 - acc: 0.9079 - val_loss: 0.3378 - val_acc: 0.8803
Epoch 20/50
Epoch 00020: val_loss improved from 0.27287 to 0.22792, saving model to best_m.h5
 - 3s - loss: 0.2185 - acc: 0.9079 - val_loss: 0.2279 - val_acc: 0.9152
Epoch 21/50
Epoch 00021: val_loss did not improve
 - 3s - loss: 

Epoch 27/50
Epoch 00027: val_loss did not improve
 - 3s - loss: 0.2389 - acc: 0.9045 - val_loss: 0.3376 - val_acc: 0.8703
Epoch 28/50
Epoch 00028: val_loss did not improve
 - 2s - loss: 0.2103 - acc: 0.9113 - val_loss: 0.8972 - val_acc: 0.6983
Epoch 29/50
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2352 - acc: 0.8944 - val_loss: 0.4435 - val_acc: 0.8080
Epoch 30/50
Epoch 00030: val_loss did not improve
 - 2s - loss: 0.2045 - acc: 0.9147 - val_loss: 0.3178 - val_acc: 0.8903
Epoch 31/50
Epoch 00031: val_loss did not improve
 - 3s - loss: 0.1944 - acc: 0.9101 - val_loss: 0.2479 - val_acc: 0.9202
Epoch 32/50
Epoch 00032: val_loss did not improve
 - 2s - loss: 0.1772 - acc: 0.9240 - val_loss: 0.2328 - val_acc: 0.9002
Epoch 33/50
Epoch 00033: val_loss did not improve
 - 2s - loss: 0.1745 - acc: 0.9273 - val_loss: 0.2405 - val_acc: 0.9002
Epoch 34/50
Epoch 00034: val_loss did not improve
 - 2s - loss: 0.1817 - acc: 0.9242 - val_loss: 0.2584 - val_acc: 0.8978
Epoch 35/50
Epoch 00035:

Epoch 42/50
Epoch 00042: val_loss did not improve
 - 2s - loss: 0.1589 - acc: 0.9383 - val_loss: 0.2079 - val_acc: 0.9077
Epoch 43/50
Epoch 00043: val_loss improved from 0.19767 to 0.19744, saving model to best_m.h5
 - 3s - loss: 0.1544 - acc: 0.9333 - val_loss: 0.1974 - val_acc: 0.9027
Epoch 44/50
Epoch 00044: val_loss improved from 0.19744 to 0.19727, saving model to best_m.h5
 - 3s - loss: 0.1494 - acc: 0.9375 - val_loss: 0.1973 - val_acc: 0.8928
Epoch 45/50
Epoch 00045: val_loss improved from 0.19727 to 0.19243, saving model to best_m.h5
 - 3s - loss: 0.1715 - acc: 0.9220 - val_loss: 0.1924 - val_acc: 0.9102
Epoch 46/50
Epoch 00046: val_loss did not improve
 - 2s - loss: 0.1658 - acc: 0.9341 - val_loss: 0.1955 - val_acc: 0.9102
Epoch 47/50
Epoch 00047: val_loss did not improve
 - 2s - loss: 0.1554 - acc: 0.9392 - val_loss: 0.2130 - val_acc: 0.8928
Epoch 48/50
Epoch 00048: val_loss did not improve
 - 2s - loss: 0.1563 - acc: 0.9347 - val_loss: 0.2623 - val_acc: 0.8828
Epoch 49/50
Ep

In [13]:
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    bn_axis = 3
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 96], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 96], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 96], stage=2, block='c')

    x = conv_block(x, 3, [96, 96, 128], stage=3, block='a')
    x = identity_block(x, 3, [96, 96, 128], stage=3, block='b')
    x = identity_block(x, 3, [96, 96, 128], stage=3, block='c')
    x = identity_block(x, 3, [96, 96, 128], stage=3, block='d')
    
    x = Conv2D(128, 3, strides=2, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [14]:

def kfold_train(fold_cnt=3,rnd=233):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            #zoom_range=0.05,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 32
        steps_train = len(curr_y)//bat_size
        
        
        model = create_resnet_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

with open('../features/resnet_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/resnet_aug1_sub.csv', index=False)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.68874, saving model to best_m.h5
 - 26s - loss: 0.9238 - acc: 0.5573 - val_loss: 0.6887 - val_acc: 0.5835
Epoch 2/50
Epoch 00002: val_loss did not improve
 - 3s - loss: 0.5520 - acc: 0.7006 - val_loss: 0.8147 - val_acc: 0.4514
Epoch 3/50
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.4886 - acc: 0.7896 - val_loss: 0.7462 - val_acc: 0.4564
Epoch 4/50
Epoch 00004: val_loss did not improve
 - 3s - loss: 0.4453 - acc: 0.8012 - val_loss: 0.9101 - val_acc: 0.6534
Epoch 5/50
Epoch 00005: val_loss did not improve
 - 3s - loss: 0.4027 - acc: 0.8260 - val_loss: 0.7248 - val_acc: 0.6658
Epoch 6/50
Epoch 00006: val_loss improved from 0.68874 to 0.67180, saving model to best_m.h5
 - 4s - loss: 0.3671 - acc: 0.8328 - val_loss: 0.6718 - val_acc: 0.7531
Epoch 7/50
Epoch 00007: val_loss did not improve
 - 3s - loss: 0.4536 - acc: 0.8070 - val_loss: 0.7121 - val_acc: 0.7207
Epoch 8/50
Epoch 00008: val_loss improved from 0.67180 to 0.47733, s

 - 5s - loss: 0.2953 - acc: 0.8612 - val_loss: 0.2552 - val_acc: 0.8953
Epoch 13/50
Epoch 00013: val_loss did not improve
 - 4s - loss: 0.2790 - acc: 0.8758 - val_loss: 0.2615 - val_acc: 0.8928
Epoch 14/50
Epoch 00014: val_loss did not improve
 - 3s - loss: 0.3081 - acc: 0.8783 - val_loss: 0.2650 - val_acc: 0.8928
Epoch 15/50
Epoch 00015: val_loss did not improve
 - 3s - loss: 0.2919 - acc: 0.8784 - val_loss: 0.2636 - val_acc: 0.8828
Epoch 16/50
Epoch 00016: val_loss improved from 0.25516 to 0.24108, saving model to best_m.h5
 - 4s - loss: 0.2349 - acc: 0.8989 - val_loss: 0.2411 - val_acc: 0.8953
Epoch 17/50
Epoch 00017: val_loss did not improve
 - 3s - loss: 0.2737 - acc: 0.8775 - val_loss: 0.2514 - val_acc: 0.9002
Epoch 18/50
Epoch 00018: val_loss did not improve
 - 4s - loss: 0.2524 - acc: 0.8877 - val_loss: 0.3318 - val_acc: 0.8828
Epoch 19/50
Epoch 00019: val_loss did not improve
 - 4s - loss: 0.2792 - acc: 0.8783 - val_loss: 0.5171 - val_acc: 0.7880
Epoch 20/50
Epoch 00020: val_l

Epoch 00025: val_loss did not improve
 - 3s - loss: 0.2484 - acc: 0.8919 - val_loss: 0.3528 - val_acc: 0.8404
Epoch 26/50
Epoch 00026: val_loss did not improve
 - 3s - loss: 0.2514 - acc: 0.8927 - val_loss: 0.2665 - val_acc: 0.8803
Epoch 27/50
Epoch 00027: val_loss did not improve
 - 3s - loss: 0.2484 - acc: 0.8983 - val_loss: 0.2789 - val_acc: 0.8803
Epoch 28/50
Epoch 00028: val_loss did not improve
 - 3s - loss: 0.2435 - acc: 0.8913 - val_loss: 0.3534 - val_acc: 0.8155
Epoch 29/50
Epoch 00029: val_loss improved from 0.23981 to 0.22744, saving model to best_m.h5
 - 4s - loss: 0.2434 - acc: 0.9054 - val_loss: 0.2274 - val_acc: 0.9127
Epoch 30/50
Epoch 00030: val_loss did not improve
 - 3s - loss: 0.2273 - acc: 0.9028 - val_loss: 0.4019 - val_acc: 0.8304
Epoch 31/50
Epoch 00031: val_loss did not improve
 - 4s - loss: 0.2113 - acc: 0.9130 - val_loss: 0.3126 - val_acc: 0.8678
Epoch 32/50
Epoch 00032: val_loss did not improve
 - 3s - loss: 0.2310 - acc: 0.9102 - val_loss: 0.2596 - val_acc:

Epoch 39/50
Epoch 00039: val_loss did not improve
 - 3s - loss: 0.2093 - acc: 0.9161 - val_loss: 0.2295 - val_acc: 0.9102
Epoch 40/50
Epoch 00040: val_loss did not improve
 - 3s - loss: 0.1853 - acc: 0.9310 - val_loss: 0.2241 - val_acc: 0.9027
Epoch 41/50
Epoch 00041: val_loss did not improve
 - 3s - loss: 0.2046 - acc: 0.9178 - val_loss: 0.2356 - val_acc: 0.9077
Epoch 42/50
Epoch 00042: val_loss did not improve
 - 3s - loss: 0.1997 - acc: 0.9158 - val_loss: 0.2610 - val_acc: 0.9102
Epoch 43/50
Epoch 00043: val_loss did not improve
 - 3s - loss: 0.2098 - acc: 0.9093 - val_loss: 0.2603 - val_acc: 0.9002
Epoch 44/50
Epoch 00044: val_loss did not improve
 - 3s - loss: 0.2347 - acc: 0.9043 - val_loss: 0.2415 - val_acc: 0.8978
Epoch 45/50
Epoch 00045: val_loss did not improve
 - 3s - loss: 0.1964 - acc: 0.9214 - val_loss: 0.2125 - val_acc: 0.9152
Epoch 46/50
Epoch 00046: val_loss did not improve
 - 3s - loss: 0.1999 - acc: 0.9026 - val_loss: 0.2343 - val_acc: 0.9102
Epoch 47/50
Epoch 00047: